In [4]:
import temgymbasic.jax_components as comp
from temgymbasic.jax_run import run_to_end
from temgymbasic.jax_ray import Ray
from temgymbasic.jax_source import PointBeam, ParallelBeam
from temgymbasic.jax_utils import calculate_phi_0
import jax.numpy as jnp
import jax


In [10]:
components = [comp.Lens(z=0.2, focal_length=0.2),
             comp.Detector(z=2.0, pixel_size=0.01, shape=(100, 100))]

n_rays = 1
n_rays_dim = int(jnp.sqrt(n_rays))

Rays = PointBeam(0.0,
                n_rays,
                semi_angle=0.,
                tilt_yx=(0, 0.01),
                centre_yx=(0., 0.),
                voltage=calculate_phi_0(1.0),
                random=False)

print(Rays)
def create_ray(ray):
    return Ray(z=ray.z, 
               matrix=ray.matrix,
               amplitude=ray.amplitude,
               pathlength=ray.pathlength,
               wavelength=ray.wavelength)

# Method 1
one_ray = jax.jit(jax.vmap(lambda r: run_to_end(create_ray(r), components)))

dro_dri = jax.jit(jax.vmap(lambda r: jax.jacobian(run_to_end, argnums=0)(create_ray(r), components)))(Rays)
dro_dmodel = jax.jit(jax.vmap(lambda r: jax.jacobian(run_to_end, argnums=1)(create_ray(r), components)))(Rays)

magnification = dro_dri.matrix.matrix[0, 0, 0]

print(magnification)

\\grenfshome01\DL277493\TemGym\src\temgymbasic\jax_utils.py:166: UserWarning: Explicitly requested dtype <class 'jax.numpy.float64'> requested in zeros is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/jax-ml/jax#current-gotchas for more.
  matrix = jnp.zeros(


Ray(matrix=Array([[0.  , 0.  , 0.01, 0.  , 1.  ]], dtype=float32), z=Array([0.], dtype=float32), amplitude=Array([1.], dtype=float32), pathlength=Array([0.], dtype=float32), wavelength=1.0, blocked=0)
-8.0


In [9]:
print(dro_dmodel.matrix)

[Lens(z=Array([[-0.08,  0.  , -0.05,  0.  ,  0.  ]], dtype=float32, weak_type=True), focal_length=Array([[0.08999999, 0.        , 0.05      , 0.        , 0.        ]],      dtype=float32, weak_type=True)), Detector(z=Array([[-6.9849193e-10,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00]], dtype=float32, weak_type=True), pixel_size=0.01, shape=(100, 100), rotation=0.0, flip_y=False, center=(0.0, 0.0))]


Move Aperture based on gradients?

Find Lens location that achieves a target magnification, while fixing focal length - Extend to Projector Lens system that can be moved around with a fixed set of focal lengths to achieve target magnification.

Align double deflector settings to find pivot point